# Import and Install Depedencies

In [3]:
!pip install keras

In [124]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pickle

# Key points using MP Holistic

In [125]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [126]:
words = ['angry','bank', 'brother','bye','excuse me','father', 'good evening', 'good morning', 'good night', 'happy','hello', 'help','home', 'hospital','how much', 'hungry','love', 'mother', 'police station', 'sad', 'school', 'sister','sorry','thankyou', 'welcome', 'what', 'when', 'where', 'who','why']
len(words)

30

In [127]:
words_2 = ['bye', 'hello', 'home', 'love', 'thankyou']

In [128]:


mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
dic = {}
with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands:
    for word in words:
        dire = os.path.join('src_2', word)
        vids = os.listdir(dire)
        vid_no = 0
        print(word)
        dic[word] = []
        for i,vid in enumerate(vids):
            dic[word].append([])
            vidcap = cv2.VideoCapture(os.path.join('src_2', word, vid))
            success,frame = vidcap.read()
            count = 0
            while success:     
                success,frame = vidcap.read()
                if success:
                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results = hands.process(image)
                    if results.multi_hand_landmarks:
                        # Loop through each hand landmark in the list
                        for hand_landmarks in results.multi_hand_landmarks:
                            # Get the landmarks for the hand
                            landmarks = hand_landmarks.landmark
                            landmarks_array = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks])
                            # Store the landmarks in the dataset array                             
                            dic[word][i].append(landmarks_array.flatten())
                            # Draw the hand landmarks on the image for visualization
                            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
            vid_no +=1
        

angry
bank
brother
bye
excuse me
father
good evening
good morning
good night
happy
hello
help
home
hospital
how much
hungry
love
mother
police station
sad
school
sister
sorry
thankyou
welcome
what
when
where
who
why


In [129]:
# create a binary pickle file 
f = open("preprocessed_all.pkl","wb")
pickle.dump(dic,f)
print("Successfuly transfered")
f.close()


Successfuly transfered


In [130]:
with open('preprocessed_all.pkl', 'rb') as f:
    data = pickle.load(f)
# print(data)
print("reading is successful")

reading is successful


In [131]:
def stretch(video, size):
    arr = np.array(video)
    n = len(arr)
    x = np.linspace(0, n - 1, n)
    new_x = np.linspace(0, n - 1, size)
    new_arr = np.zeros((size, len(video[0])))
    for i in range(size):
        new_arr[:, i] = np.interp(new_x, x, arr[:, i])
    
    return new_arr

In [132]:
# l = stretch(data['bye'][0], 63)
# print(np.array(l).shape)
# print(l)
# print(data['bye'][0])

In [133]:
for key in data.keys():
    print(key, len(data[key][1]))

angry 17
bank 109
brother 32
bye 38
excuse me 65
father 31
good evening 56
good morning 65
good night 74
happy 52
hello 34
help 60
home 19
hospital 103
how much 10
hungry 21
love 41
mother 30
police station 118
sad 27
school 75
sister 24
sorry 51
thankyou 23
welcome 35
what 17
when 29
where 9
who 19
why 24


In [134]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

src_2 = {} 
for key in data.keys():
    arr = data[key]
    src_2[key] = []
    for vid in arr:
        if len(vid) !=0 :
            src_2[key].append(stretch(vid, 63))

In [135]:
minn = 200
for arr in src_2.values():
    for ar in arr:
        minn = min(minn, len(ar))
        
print(minn)

63


In [136]:
# for key in src_2.keys():
#     print(key, len(src_2[key]))

In [137]:
src_3 = {}
for key in src_2.keys():
    if len(src_2[key]) <= 63 and len(src_2[key]) >= 10:
        src_3[key] = src_2[key]

In [138]:
len(src_3['bye'][0][1])

63

In [139]:
for key in src_3.keys():
    print(key, len(src_3[key]))

angry 25
bank 25
brother 25
bye 25
excuse me 25
father 25
good evening 25
good morning 25
good night 25
happy 25
hello 25
help 25
home 25
hospital 25
how much 25
hungry 25
love 25
mother 25
police station 25
sad 25
school 25
sister 25
sorry 25
thankyou 25
welcome 25
what 25
when 25
where 25
who 25
why 25


In [140]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [141]:
words = src_2.keys()

In [142]:
label_map = {label:num for num, label in enumerate(words)}

In [143]:
label_map

{'angry': 0,
 'bank': 1,
 'brother': 2,
 'bye': 3,
 'excuse me': 4,
 'father': 5,
 'good evening': 6,
 'good morning': 7,
 'good night': 8,
 'happy': 9,
 'hello': 10,
 'help': 11,
 'home': 12,
 'hospital': 13,
 'how much': 14,
 'hungry': 15,
 'love': 16,
 'mother': 17,
 'police station': 18,
 'sad': 19,
 'school': 20,
 'sister': 21,
 'sorry': 22,
 'thankyou': 23,
 'welcome': 24,
 'what': 25,
 'when': 26,
 'where': 27,
 'who': 28,
 'why': 29}

In [144]:
src_3['bye'][0][0][0]

0.39737629890441895

In [145]:
sequences, labels = [], []
for word in words:
    for i,sequence in enumerate(src_3[word]):
        window = src_3[word][i][:63]
        sequences.append(np.asarray(window).astype('float32'))
        labels.append(label_map[word])

In [146]:
np.array(sequences).shape

(750, 63, 63)

In [147]:
np.array(labels).shape

(750,)

In [148]:
X = np.array(sequences)

In [149]:
X.shape

(750, 63, 63)

In [150]:
y = to_categorical(labels).astype(int)

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [152]:
y_test.shape

(150, 30)

# Build and Train LSTM Neural Network

In [153]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [154]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [155]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(63,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(words), activation='softmax'))

In [156]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [158]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
19/19 [==============================] - 2s 104ms/step - loss: 1.0525 - categorical_accuracy: 0.6167
Epoch 2/2000
19/19 [==============================] - 2s 101ms/step - loss: 0.9484 - categorical_accuracy: 0.6533
Epoch 3/2000
19/19 [==============================] - 2s 109ms/step - loss: 0.9926 - categorical_accuracy: 0.6367
Epoch 4/2000
19/19 [==============================] - 2s 102ms/step - loss: 0.9803 - categorical_accuracy: 0.6483
Epoch 5/2000
19/19 [==============================] - 2s 104ms/step - loss: 0.8777 - categorical_accuracy: 0.6867
Epoch 6/2000
19/19 [==============================] - 2s 102ms/step - loss: 0.8813 - categorical_accuracy: 0.6750
Epoch 7/2000
19/19 [==============================] - 2s 104ms/step - loss: 0.8131 - categorical_accuracy: 0.6983
Epoch 8/2000
19/19 [==============================] - 2s 101ms/step - loss: 0.7623 - categorical_accuracy: 0.7200
Epoch 9/2000
19/19 [==============================] - 2s 106ms/step - loss: 0.7580 - cat

19/19 [==============================] - 2s 99ms/step - loss: 0.2364 - categorical_accuracy: 0.9167
Epoch 73/2000
19/19 [==============================] - 2s 100ms/step - loss: 0.2191 - categorical_accuracy: 0.9283
Epoch 74/2000
19/19 [==============================] - 2s 102ms/step - loss: 0.2127 - categorical_accuracy: 0.9200
Epoch 75/2000
19/19 [==============================] - 2s 105ms/step - loss: 0.2133 - categorical_accuracy: 0.9133
Epoch 76/2000
19/19 [==============================] - 2s 101ms/step - loss: 0.1914 - categorical_accuracy: 0.9350
Epoch 77/2000
19/19 [==============================] - 2s 104ms/step - loss: 0.2146 - categorical_accuracy: 0.9233
Epoch 78/2000
19/19 [==============================] - 2s 106ms/step - loss: 0.1860 - categorical_accuracy: 0.9350
Epoch 79/2000
19/19 [==============================] - 2s 96ms/step - loss: 0.2203 - categorical_accuracy: 0.9333
Epoch 80/2000
19/19 [==============================] - 2s 100ms/step - loss: 0.2982 - categorica

19/19 [==============================] - 2s 106ms/step - loss: 0.2425 - categorical_accuracy: 0.9283
Epoch 144/2000
19/19 [==============================] - 2s 106ms/step - loss: 0.1468 - categorical_accuracy: 0.9533
Epoch 145/2000
19/19 [==============================] - 2s 102ms/step - loss: 0.1193 - categorical_accuracy: 0.9567
Epoch 146/2000
19/19 [==============================] - 2s 102ms/step - loss: 0.1200 - categorical_accuracy: 0.9650
Epoch 147/2000
19/19 [==============================] - 2s 106ms/step - loss: 0.0985 - categorical_accuracy: 0.9683
Epoch 148/2000
19/19 [==============================] - 2s 107ms/step - loss: 0.1226 - categorical_accuracy: 0.9567
Epoch 149/2000
19/19 [==============================] - 2s 104ms/step - loss: 0.3859 - categorical_accuracy: 0.8883
Epoch 150/2000
19/19 [==============================] - 2s 101ms/step - loss: 0.9726 - categorical_accuracy: 0.7267
Epoch 151/2000
19/19 [==============================] - 2s 103ms/step - loss: 0.5802 - 

19/19 [==============================] - 2s 104ms/step - loss: 0.0372 - categorical_accuracy: 0.9950
Epoch 214/2000
19/19 [==============================] - 2s 102ms/step - loss: 0.0337 - categorical_accuracy: 0.9933
Epoch 215/2000
19/19 [==============================] - 2s 105ms/step - loss: 0.0336 - categorical_accuracy: 0.9950
Epoch 216/2000
19/19 [==============================] - 2s 103ms/step - loss: 0.0234 - categorical_accuracy: 0.9983
Epoch 217/2000
19/19 [==============================] - 2s 105ms/step - loss: 0.0254 - categorical_accuracy: 0.9967
Epoch 218/2000
19/19 [==============================] - 2s 100ms/step - loss: 0.0271 - categorical_accuracy: 0.9933
Epoch 219/2000
19/19 [==============================] - 2s 105ms/step - loss: 0.0250 - categorical_accuracy: 0.9967
Epoch 220/2000
19/19 [==============================] - 2s 106ms/step - loss: 0.0372 - categorical_accuracy: 0.9883
Epoch 221/2000
19/19 [==============================] - 2s 104ms/step - loss: 0.0252 - 

19/19 [==============================] - 1s 78ms/step - loss: 0.7337 - categorical_accuracy: 0.7433
Epoch 285/2000
19/19 [==============================] - 2s 79ms/step - loss: 0.7495 - categorical_accuracy: 0.7233
Epoch 286/2000
19/19 [==============================] - 1s 79ms/step - loss: 0.7107 - categorical_accuracy: 0.7383
Epoch 287/2000
19/19 [==============================] - 1s 78ms/step - loss: 0.6797 - categorical_accuracy: 0.7450
Epoch 288/2000
19/19 [==============================] - 3s 143ms/step - loss: 0.6383 - categorical_accuracy: 0.7617
Epoch 289/2000
19/19 [==============================] - 3s 167ms/step - loss: 0.7474 - categorical_accuracy: 0.7300
Epoch 290/2000
19/19 [==============================] - 3s 162ms/step - loss: 0.6546 - categorical_accuracy: 0.7583
Epoch 291/2000
19/19 [==============================] - 2s 86ms/step - loss: 0.6330 - categorical_accuracy: 0.7600
Epoch 292/2000
19/19 [==============================] - 1s 77ms/step - loss: 0.5251 - catego

...

19/19 [==============================] - 2s 81ms/step - loss: 0.2117 - categorical_accuracy: 0.9333
Epoch 1688/2000
19/19 [==============================] - 2s 118ms/step - loss: 0.2121 - categorical_accuracy: 0.9333
Epoch 1689/2000
19/19 [==============================] - 19s 145ms/step - loss: 0.1599 - categorical_accuracy: 0.9483
Epoch 1690/2000
19/19 [==============================] - 2s 111ms/step - loss: 0.1691 - categorical_accuracy: 0.9550
Epoch 1691/2000
19/19 [==============================] - 2s 114ms/step - loss: 0.1221 - categorical_accuracy: 0.9617
Epoch 1692/2000
19/19 [==============================] - 3s 166ms/step - loss: 0.0917 - categorical_accuracy: 0.9800
Epoch 1693/2000
19/19 [==============================] - 5s 279ms/step - loss: 0.0801 - categorical_accuracy: 0.9917
Epoch 1694/2000
19/19 [==============================] - 4s 212ms/step - loss: 0.0771 - categorical_accuracy: 0.9833
Epoch 1695/2000
19/19 [==============================] - 4s 218ms/step - loss: 0

19/19 [==============================] - 3s 136ms/step - loss: 2.2692 - categorical_accuracy: 0.4067
Epoch 1758/2000
19/19 [==============================] - 3s 131ms/step - loss: 1.4544 - categorical_accuracy: 0.5300
Epoch 1759/2000
19/19 [==============================] - 2s 113ms/step - loss: 1.2279 - categorical_accuracy: 0.6117
Epoch 1760/2000
19/19 [==============================] - 2s 117ms/step - loss: 1.0501 - categorical_accuracy: 0.6517
Epoch 1761/2000
19/19 [==============================] - 2s 112ms/step - loss: 0.9430 - categorical_accuracy: 0.6733
Epoch 1762/2000
19/19 [==============================] - 2s 114ms/step - loss: 0.9001 - categorical_accuracy: 0.6933
Epoch 1763/2000
19/19 [==============================] - 2s 115ms/step - loss: 0.8335 - categorical_accuracy: 0.7100
Epoch 1764/2000
19/19 [==============================] - 2s 116ms/step - loss: 0.7736 - categorical_accuracy: 0.7467
Epoch 1765/2000
19/19 [==============================] - 2s 119ms/step - loss: 0

19/19 [==============================] - 2s 114ms/step - loss: 0.2430 - categorical_accuracy: 0.9333
Epoch 1828/2000
19/19 [==============================] - 2s 117ms/step - loss: 0.6060 - categorical_accuracy: 0.8217
Epoch 1829/2000
19/19 [==============================] - 2s 117ms/step - loss: 0.4474 - categorical_accuracy: 0.8600
Epoch 1830/2000
19/19 [==============================] - 2s 117ms/step - loss: 0.3568 - categorical_accuracy: 0.8733
Epoch 1831/2000
19/19 [==============================] - 2s 117ms/step - loss: 0.1993 - categorical_accuracy: 0.9317
Epoch 1832/2000
19/19 [==============================] - 2s 113ms/step - loss: 0.1545 - categorical_accuracy: 0.9583
Epoch 1833/2000
19/19 [==============================] - 2s 117ms/step - loss: 0.1267 - categorical_accuracy: 0.9733
Epoch 1834/2000
19/19 [==============================] - 2s 116ms/step - loss: 0.1100 - categorical_accuracy: 0.9767
Epoch 1835/2000
19/19 [==============================] - 2s 115ms/step - loss: 0

19/19 [==============================] - 2s 116ms/step - loss: 0.0456 - categorical_accuracy: 0.9917
Epoch 1898/2000
19/19 [==============================] - 2s 115ms/step - loss: 0.0366 - categorical_accuracy: 0.9917
Epoch 1899/2000
19/19 [==============================] - 2s 117ms/step - loss: 0.0380 - categorical_accuracy: 0.9967
Epoch 1900/2000
19/19 [==============================] - 2s 118ms/step - loss: 0.0374 - categorical_accuracy: 0.9950
Epoch 1901/2000
19/19 [==============================] - 2s 113ms/step - loss: 0.0326 - categorical_accuracy: 0.9950
Epoch 1902/2000
19/19 [==============================] - 2s 118ms/step - loss: 0.0326 - categorical_accuracy: 0.9950
Epoch 1903/2000
19/19 [==============================] - 2s 116ms/step - loss: 0.0400 - categorical_accuracy: 0.9933
Epoch 1904/2000
19/19 [==============================] - 2s 116ms/step - loss: 0.0316 - categorical_accuracy: 0.9983
Epoch 1905/2000
19/19 [==============================] - 2s 116ms/step - loss: 0

19/19 [==============================] - 2s 115ms/step - loss: 0.0178 - categorical_accuracy: 0.9983
Epoch 1968/2000
19/19 [==============================] - 2s 116ms/step - loss: 0.0194 - categorical_accuracy: 0.9983
Epoch 1969/2000
19/19 [==============================] - 2s 114ms/step - loss: 0.0165 - categorical_accuracy: 1.0000
Epoch 1970/2000
19/19 [==============================] - 2s 114ms/step - loss: 0.0157 - categorical_accuracy: 1.0000
Epoch 1971/2000
19/19 [==============================] - 2s 117ms/step - loss: 0.0143 - categorical_accuracy: 1.0000
Epoch 1972/2000
19/19 [==============================] - 2s 115ms/step - loss: 0.0136 - categorical_accuracy: 1.0000
Epoch 1973/2000
19/19 [==============================] - 2s 113ms/step - loss: 0.0140 - categorical_accuracy: 1.0000
Epoch 1974/2000
19/19 [==============================] - 2s 120ms/step - loss: 0.0131 - categorical_accuracy: 1.0000
Epoch 1975/2000
19/19 [==============================] - 2s 114ms/step - loss: 0

In [162]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 63, 64)            32768     
                                                                 
 lstm_13 (LSTM)              (None, 63, 128)           98816     
                                                                 
 lstm_14 (LSTM)              (None, 64)                49408     
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 30)                990       
                                                                 
Total params: 188,222
Trainable params: 188,222
Non-tr

# Make Predictions

In [163]:
res = model.predict(X_test)

5/5 [==============================] - 0s 45ms/step


In [164]:
X_test.shape

(150, 63, 63)

# Save Weights

# Evaluation using Confusion Matrix and Accuracy

In [165]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [166]:
yhat = model.predict(X_test)

5/5 [==============================] - 0s 43ms/step


In [167]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [168]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[141,   2],
        [  4,   3]],

       [[144,   2],
        [  2,   2]],

       [[144,   1],
        [  1,   4]],

       [[142,   3],
        [  2,   3]],

       [[147,   0],
        [  0,   3]],

       [[147,   0],
        [  0,   3]],

       [[138,   2],
        [  2,   8]],

       [[146,   0],
        [  0,   4]],

       [[144,   1],
        [  3,   2]],

       [[147,   1],
        [  0,   2]],

       [[146,   0],
        [  0,   4]],

       [[140,   1],
        [  5,   4]],

       [[145,   1],
        [  2,   2]],

       [[145,   0],
        [  2,   3]],

       [[141,   2],
        [  1,   6]],

       [[143,   0],
        [  0,   7]],

       [[146,   3],
        [  1,   0]],

       [[144,   0],
        [  0,   6]],

       [[146,   1],
        [  0,   3]],

       [[140,   2],
        [  0,   8]],

       [[148,   2],
        [  0,   0]],

       [[144,   1],
        [  0,   5]],

       [[149,   0],
        [  0,   1]],

       [[141,   1],
        [  1, 

In [169]:
accuracy_score(ytrue, yhat)

0.74

In [170]:
with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands:
    vidcap = cv2.VideoCapture('love.mp4')
    success,frame = vidcap.read()
    count = 0
    test = []
    while success:     
        success,frame = vidcap.read()
        if success:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = hand_landmarks.landmark
                    landmarks_array = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks])
                    test.append(landmarks_array.flatten())


In [171]:
nptest = np.array([test[:35]])

In [172]:
nptest.shape

(1, 25, 63)

In [173]:
z = model.predict(nptest)

1/1 [==============================] - 0s 321ms/step


In [174]:
words[np.argmax(z[0])]

TypeError: 'dict_keys' object is not subscriptable

In [177]:
pip show scikit-learn

Name: scikit-learn
Version: 1.0.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: c:\users\jathusharini\anaconda3\lib\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: scikit-learn-intelex
Note: you may need to restart the kernel to use updated packages.


In [180]:
pip show tensorflow

Name: tensorflow
Version: 2.11.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\jathusharini\anaconda3\lib\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.
